In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
from PIL import Image
import base64
import re
import numpy as np
import cv2
import time
def changeImage(dataUrl):
    dataUrl = re.sub('^data:image/.+;base64,','',dataUrl)
    image_s = base64.b64decode(dataUrl)
    fileName ='test.jpg'
    file= open(fileName,'wb')  
    file.write(image_s)  
    file.close()
    img = Image.open('./test.jpg').convert('L')
    if img.size[0] != 28 or img.size[1] != 28:
        img = img.resize((28, 28))
    arr = []
    for i in range(28):
        for j in range(28):
            pixel = 1.0 - float(img.getpixel((j, i)))/255.0
            arr.append(pixel)
    arr1 = np.array(arr).reshape((1,784))
    print(arr1)
    return arr1

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.001 #学习率 
batch_size = 128 #批尺
num_steps = 100 #使用的样本数量
display_step = 10 #显示间隔

num_input = 784 # MNIST data input (image shape:28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 #随机丢弃一些数据 防止或者减轻拟合

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)


In [3]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
    

In [4]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
pred = tf.argmax(prediction, 1)
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
saver = tf.train.Saver(tf.trainable_variables())

In [5]:
def train():
    sess.run(init)
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss={:.4f}".format(loss) + ", Training Accuracy={:.3f}".format(acc))
    # Calculate accuracy for 256 MNIST test images
    print(sess.run(accuracy, feed_dict={X: mnist.test.images[:500],
                                      Y: mnist.test.labels[:500],
                                      keep_prob: 1.0}))
    save_path = saver.save(sess,"../tmp/model.ckpt")
    print("save success in:"+save_path)

In [6]:
#load model and prediction
with tf.Session() as sess:
#     train()
    saver.restore(sess, "../tmp/model.ckpt")
    arr1 = changeImage("data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAYAAAByDd+UAAACvUlEQVRIS+1WPUiyURg9LhYhEUImStqgU2Au4WJE0GCp4NAQBZqQhSCUCSq52B85GDg4OjSUNCX2Q6M4uClESK4SIQT9OBQRQX48FxTD6r19RnzDd8FF7/uce84957yKarVaDb+4RP8Bf1rtf1vScrmM3d1d9PT0oFqtMvJisRhdXV3o7e3F7OysoCDcDMPhME5OTlAoFDA6OopsNsuG9/X1QaPR4O3tDYuLi3A4HF+CcgEeHBwgGo3i5eUFZrMZcrn8HcPX11ckEgkGnMlk2gfc3t7G9fU1JiYmYLFYWgbe3d1hfHwc3d3dDeafoQoyPD8/x8rKCrurqakpzM/Pt8wiqa1WK4aGhkD7v1qCgGQOl8uFi4sLPD8/Q6fTYW9vjxmnvshITqcTS0tLiMVi7QHWnyYWwWAQdF8ymQzHx8cNUJVKBa1WC6/X+6HkzScQZNi8mdiaTCbc3t6iv7+fRWRnZwdnZ2cYHh5GMpn8uVjUJxHoyMgIisUiJBIJHh8fGauNjQ3o9fqfByRTbG1tIZVKNXJ4enrKBUYPcEtKzNbW1ljOnp6eUKlUoFarWfaOjo4EmdU3cAHu7+/D4/Ews1CNUdOQQebm5vDw8IB4PA6j0cgFKghIQMTq8vISMzMzTM6BgQE2fHl5Gfl8nh1menq6fcD19XXmQrL8wsIC+zQvMsrh4SECgUD7gPRmINlKpRKoSz9yIEVEJBIhFAq1LymVNd0dlfXm5maLXHQgisfg4CA7UHPz/FW1UahXV1dZoNPpdMuMsbEx9p3BYEAkEuG6P8FYKBQKKJVK+Hy+d3dE7szlcujs7GQVVzcRD+qXLqUiJqb39/csa5RFu93O3hxUb/SbzWbjwWnsEYyF2+1uhL2jowNSqZSF3e/3c7fLt8ubmN7c3ODq6gqTk5Nc/10+oy3I8Ft6cWz+dcA/8zN3vADs60gAAAAASUVORK5CYII=")
# #     print(sess.run(accuracy, feed_dict={X: mnist.test.images[:500],
# #                                       Y: mnist.test.labels[:500],
# #                                       keep_prob: 1.0}))
    print(sess.run(pred,feed_dict={X:arr1,keep_prob: 1.0}))

INFO:tensorflow:Restoring parameters from ../tmp/model.ckpt
[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0